In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

从2024年9月15日开始，您只能通过转移到Vertex AI Gemini提示和调整来定制分类、实体提取和情感分析模型。对于Vertex AI AutoML文本分类、实体提取和情感分析目标的训练或更新模型将不再可用。您可以继续使用现有的Vertex AI AutoML文本目标，直到2025年6月15日。有关Gemini如何通过改进的提示功能提供增强用户体验的更多信息，请参阅[调整简介](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-gemini-overview)。

# Vertex AI 迁移：AutoML 文本分类

<table align="left">
  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
      <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>   
</table>
<br/><br/><br/>

## 概览

本笔记本演示了如何使用Vertex AI文本数据集创建AutoML文本分类模型，以及如何为批量预测提供该模型服务。您需要提供一个存储数据集的存储桶。

注：在测试此SDK时，您可能会因训练、预测、存储或使用其他GCP产品而产生费用。

了解更多关于[迁移到 Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[文本数据分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_text)。

### 目标

本笔记本的目标是构建一个自动机文本分类模型。已经按照以下步骤进行操作：
本教程使用以下 Google Cloud ML 服务：

* Vertex AI 数据集资源
* AutoML 训练
* Vertex AI 模型资源
* Vertex AI 批量预测

所执行的步骤包括以下内容：

* 设置您的任务名称和 GCS 前缀
* 复制 AutoML 文本演示训练数据以创建托管数据集
* 在 Vertex AI 上创建数据集。
* 配置一个训练作业
* 启动训练作业并在 Vertex AI 上创建模型
* 复制 AutoML 文本演示预测数据以创建批量预测作业
* 对模型执行批量预测作业

数据集

本教程使用的数据集是来自Kaggle数据集的[快乐时刻数据集](https://www.kaggle.com/ritresearch/happydb)。您在本教程中将使用的数据集版本存储在公共云存储桶中。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成成本估算。

## 安装

安装执行该笔记本所需的包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow 

### 仅限使用Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。 了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免资源创建时用户之间的名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google云账户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要操作，因为您已经通过验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他  
* 请查看如何给予您的服务账号云存储权限的方法，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速浏览数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来统计示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.TextDataset.create(
    display_name="Happy Moments" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

示例输出：

    INFO:google.cloud.aiplatform.datasets.dataset:创建TextDataset
    INFO:google.cloud.aiplatform.datasets.dataset:创建TextDataset支持的LRO：projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/3193181053544038400
    INFO:google.cloud.aiplatform.datasets.dataset:TextDataset创建完成。资源名称：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    INFO:google.cloud.aiplatform.datasets.dataset:要在另一个会话中使用此TextDataset：
    INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TextDataset('projects/759209241365/locations/us-central1/datasets/3704325042721521664')
    INFO:google.cloud.aiplatform.datasets.dataset:正在导入TextDataset数据：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    INFO:google.cloud.aiplatform.datasets.dataset:导入TextDataset数据支持的LRO：projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/5152246891450204160
    INFO:google.cloud.aiplatform.datasets.dataset:TextDataset数据已导入。资源名称：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    projects/759209241365/locations/us-central1/datasets/3704325042721521664

## 训练模型

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行这个管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，包括以下参数：

- `display_name`: `TrainingJob`资源的可读名称。
- `prediction_type`: 为模型训练指定的任务类型。
  - `classification`: 文本分类模型。
  - `sentiment`: 文本情感分析模型。
  - `extraction`: 文本实体提取模型。
- `multi_label`: 如果是分类任务，指定是单标签（False）还是多标签（True）。
- `sentiment_max`: 如果是情感分析任务，指定最大情感值。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTextTrainingJob(
    display_name="happydb_" + UUID,
    prediction_type="classification",
    multi_label=False,
)

print(dag)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLTextTrainingJob 对象在 0x7fc3b6c90f10>

运行训练管道

接下来，您可以运行DAG来启动训练作业，通过调用方法 `run`，并传入以下参数：

- `dataset`：用于训练模型的 `Dataset` 资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留存数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

完成时，`run` 方法将返回 `Model` 资源。

训练管道的执行将需要几个小时。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

示例输出：

    INFO:google.cloud.aiplatform.training_jobs:查看训练:
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/8859754745456230400?project=759209241365
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 当前状态:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 当前状态:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 当前状态:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 当前状态:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 当前状态:
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob 运行完成。资源名称: projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400
    INFO:google.cloud.aiplatform.training_jobs:模型可在 projects/759209241365/locations/us-central1/models/6389525951797002240 中找到.

# 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。和数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=happydb_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

示例输出：

    名称：“projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824”
    metrics_schema_uri：“gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml”
    metrics {
      struct_value {
        fields {
          key: “auPrc”
          value {
            number_value: 0.9891107
          }
        }
        fields {
          key: “confidenceMetrics”
          value {
            list_value {
              values {
                struct_value {
                  fields {
                    key: “precision”
                    value {
                      number_value: 0.2
                    }
                  }
                  fields {
                    key: “recall”
                    value {
                      number_value: 1.0
                    }
                  }
                }
              }

## 进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### 获取测试项

现在对您的 Vertex 模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不必担心这些示例可能已用于训练模型 -- 您只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2
if len(test_items[0]) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 制作批量输入文件

现在制作一个批量输入文件，您将把它存储在您的本地云存储桶中。批量输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）在每行中制作一个字典条目。字典包含键/值对：

- `content`：文本项目文件的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个`text`文件。

例如：

                        {'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json
import os

import tensorflow as tf

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测任务的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，则调用将在等待异步批量任务完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="happydb_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

示例输出：

    INFO：google.cloud.aiplatform.jobs：创建批量预测作业
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖项完成。
    INFO：google.cloud.aiplatform.jobs：已创建批量预测作业。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO：google.cloud.aiplatform.jobs：要在另一个会话中使用此批量预测作业：
    INFO：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob（'projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296'）
    INFO：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO：google.cloud.aiplatform.jobs：批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：
    JobState.JOB_STATE_RUNNING

### 等待批处理预测作业完成

接下来，请等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

示例输出：

    信息：google.cloud.aiplatform.jobs：创建了BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    信息：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    信息：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    信息：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob 运行完成。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果被写入了您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法来获取生成结果的每个云存储文件的列表。每个文件都以JSON格式包含一个或多个预测请求：

- `content`: 预测请求。
- `prediction`: 预测响应。
 - `ids`: 每个预测请求的内部分配的唯一标识符。
 - `displayNames`: 每个类标签的类名。
 - `confidences`: 每个类标签的预测置信度，介于0和1之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

*示例输出:*

{'实例': {'内容': 'gs://andy-1234-221921aip-20210803210202/test2.txt', 'MIME 类型': 'text/plain'}, '预测': {'ids': ['681905016918769664', '3564208778435887104', '8175894796863275008', '5538192790107717632', '5870051787649581056', '3232349780894023680', '926506771680329728'], '展示名称': ['情感', '成就', '联结', '享受时刻', '自然', '消遣', '锻炼'], '置信度': [0.9977309, 0.0017838771, 0.0002530971, 0.00014939539, 4.747714e-05, 2.6297073e-05, 8.965492e-06]}}

进行在线预测

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

部署模型

接下来，将您的模型部署用于在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

示例输出：

    INFO:google.cloud.aiplatform.models:创建端点
    INFO:google.cloud.aiplatform.models:创建端点后台 LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    INFO:google.cloud.aiplatform.models:端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:要在另一个会话中使用此端点：
    INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    INFO:google.cloud.aiplatform.models:将模型部署到端点: projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:部署端点模型后台 LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    INFO:google.cloud.aiplatform.models:端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

## 自动机器学习的在线预测
### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### 获取测试项目

您将从数据集中选择一个任意的示例作为测试项目。不需要担心该示例很可能已经在训练模型时使用过 - 您只需展示如何进行预测即可。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在你的`Model`资源已部署到一个`Endpoint`资源中，你可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式如下：

     { 'content': text_string }

由于predict()方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从predict()调用的响应是一个Python字典，其中包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类标签的类名。
- `confidences`：每个类别标签的预测置信度，介于0到1之间。
- `deployed_model_id`：执行预测的部署的`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

*Example output:*

预测（预测= [ {'置信度': [0.9807877540588379, 0.0029202369041740894, 0.001903864904306829, 0.013396155089139938, 0.0002868965675588697, 0.00017845185357145965, 0.0005265594809316099], '显示名称': ['疼爱', '成就', '享受当下', '联系', '休闲', '自然', '锻炼'], 'ID': ['1022137895017775104', '6210284665748586496', '3327980904231469056', '8516127674962280448', '7939666922658856960', '3904441656534892544', '5633823913445163008']}], 部署模型ID='8719822099612434432', 说明=None）
疼爱

取消部署模型

当您完成预测时，您可以从“端点”资源中取消部署模型。这将取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除为本教程使用的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete the bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI